# SKYFAST: Tutorial

This tutorial demonstrates how to use SKYFAST (Demasi et al., in preparation), a Python code designed for the rapid localization of gravitational wave hosts. 

SKYFAST takes posterior samples produced by a parameter estimation pipeline as input and uses them to reconstruct an analytic joint posterior for right ascension ($\alpha$), declination ($\delta$), luminosity distance ($d_L$), and, optionally, the inclination angle ($\theta_{jn}$) of the GW event. 

SKYFAST is based on FIGARO ([Rinaldi & Del Pozzo (2024)](https://joss.theoj.org/papers/10.21105/joss.06589)), a [publicly available](https://github.com/sterinaldi/FIGARO.git) inference code designed to estimate multivariate probability densities given samples from an unknown distribution using a DDirichlet Process Gaussian Mixture Model (DPGMM) as non-parameteric model. 


## Installation

We recomment to create a conda environment with Python 3.11:

`conda create --name skyfast_env python==3.11`.


Then activate it:

`conda activate skyfast_env`


Lastly, download SKYFAST from git and install it:

`git clone git@github.com:gabrieledemasi/skyfast.git`

`cd skyfast`

`pip install .`





## Using SKYFAST to rank the most probable galaxy hosts of a GW event

In the first part of this tutorial, we show one of the main functionalities of SKYFAST, which is galaxy host ranking. 

As a working example, we look for the most probable galaxy host for the event GW170817 within the GLADE+ galaxy catalog.

### Step 1: load the GLADE+ galaxy catalog

In this example, we use SKYFAST to find the most probable galaxy hosts within the GLADE+ catalog ([Dálya et al. 2022](https://arxiv.org/abs/2110.06184)). However, the pipeline can be easily adapted to any catalog.

Download the "GLADE+.txt' file from http://glade.elte.hu/ (should be around 6 GB)
 
Preprocess it by running: 

`make_glade -i "/path_to_glade_file_position/GLADE+.txt"` 

This command creates in the same folder a lighter glade+.hdf5 file, that can be passed to SKYFAST, in the same directory.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from skyfast.skyfast import skyfast
from figaro.utils import get_priors
import h5py
from tqdm import tqdm

glade_file = '/path_to_glade_file_position/glade+.hdf5' #raplace this with the correct path!!

### Step 2: load the posterior samples

We will use the posterior samples for GW170817 from Finstad et al. 2018 (https://arxiv.org/abs/1804.04179), publicly available [here](https://github.com/sugwg/gw170817-inclination-angle). 

In principle, SKYFAST can use the posterior samples produced by any PE pipeline, as long as they are arranged as an array with as many rows as the number of samples and either 3 or 4 columns, depending on whether we are including the inclination angle or not. Angles must be expressed in radians, luminosity distance in Mpc.

In [ ]:
#load samples from the .hdf5 file
with h5py.File('data/gw_only_posteriors.hdf', 'r') as file:
    ra = np.array(file['samples']['ra'])
    dec = np.array(file['samples']['dec'])
    d_L = np.array(file['samples']['distance'])
    theta_jn = np.array(file['samples']['inclination']) 

#arrange them in a format suitable for SKYFAST 
samples = np.array([ra, dec, d_L, theta_jn]).T

#The file contains a lot of samples, so we take only one third of them for the sake of time
samples = samples[::3]
np.random.shuffle(samples)


### Step 3: compute priors for the DPGMM

We use the samples to set reasonable priors for the mean and variance of each Gaussian component in the mixture with the figaro.utils method `get_priors`. 

In [ ]:
max_dist = np.max(np.array(samples)[:,2])+1
bounds = np.array([[0., 2*np.pi], [-np.pi/2, np.pi/2], [0., max_dist], [0, np.pi]])
prior_pars = get_priors(bounds = bounds, samples = samples, probit = True)

### Step 4: Initialize SKYFAST 

We are now ready to initialize an instance of SKYFAST, which we will use to reconstruct the position and inclination angle posteriors for GW170817. It should take only a bunch of seconds. 

In [ ]:
dens = skyfast(max_dist,                                   #maximum distance for the grid
                glade_file=glade_file,                     #path to glade+.hdf5 file created before
                prior_pars= prior_pars,                    #priors for the DPGMM
                n_gridpoints=[800, 400, 100],              #points in the 3D coordinate grid (ra, dec, dL)
                inclination = True,                        #we want to include the inclination angle in the reconstruction
                true_host=[3.44613, -0.40812, 41.00982],   #position of NGC 4993, the true host of GW170817
                true_inclination = [2.5831],               #GW170817 inclunation angle, as reported in Finstad+18     
                out_name = 'gw170817_ranking_test',        #name of the output from the current analysis
                out_folder= './results_tutorial',          #path to the output folder
                theta_condition = True                     #we want to compute the inclination angle posterior conditioned to the position of each potential galaxy host  
                )

### Step 5: posterior reconstruction 

Now, we pass the samples one by one to SKYFAST, which uses them to reconstruct an analytical joint posterior distribution for the parameters $\{ \alpha, \delta, d_\text{L}, \theta_{jn} \}$.

In [ ]:
# Pass samples one by one
for i in tqdm(range(len(samples))):
        dens.intermediate_skymap(samples[i])

# Compute the posterior as a mixture of gaussians
dens.density = dens.mix.build_mixture()        

### Step 6: save all the outputs

After all the samples are processed, the posterior reconstruction, along with the related skymap, volume map, and list of the most probable galaxy hosts, are released and saved in the output directory, marked as 'final'. It is possible to visualize the plots using show = True; otherwise, they will be saved without displaying.

We also save a log file with all the key information of the current SKYFAST run.  

In [ ]:
# Save reconstructed posterior
dens.save_density(final_map = True)

# Corner plot
dens.plot_samples(samples, final_map = True, show = True)

# Skymap 
dens.make_skymap(final_map = True, show = True)

# Volume map and ranked list of most probable galaxy hosts 
dens.make_volume_map(final_map = True,  show = True)

# Log file
dens.save_log() 

### Step 7: return to initial conditions 

It is good practice to initialize SKYFAST at the initial conditions at the end of each run; otherwise, some variables could remain stored and affect the results of the subsequent run. 

This is particularly important when using SKYFAST in parallel with a PE algorithm, where it is necessary to run SKYFAST repeatedly on the same event as new samples are produced.

In [ ]:
dens.initialise()

## Using SKYFAST in parallel with a PE algorithm

An important feature of SKYFAST is its ability to be used in parallel with a PE algorithm, provided that the posterior samples are released during the run (as is the case for most MCMC samplers, for examples). As we saw in the first part of the tutorial, the samples are passed one by one to SKYFAST. Therefore, it is possible to process the samples as soon as they are produced by the PE algorithm.

If we set `entropy=True` at initialization, SKYFAST computes the information entropy of the reconstructed posterior each time a sample is added to the DPGMM mixture.

When the information entropy reaches a plateau, adding further samples does not provide much additional information. This convergence criterion is usually met using only a fraction of the samples produced by the PE, resulting in considerable time savings.

In the second part of this tutorial, we will demonstrate this functionality of SKYFAST using the same samples for GW170817. We will compute all the outputs (skymap, volume map, ranked list) for both the intermediate reconstruction, obtained soon after convergence is reached, and the final reconstruction, obtained using all the samples, which is essentially what we obtained in the first part of the tutorial. We will verify that the intermediate reconstruction is nearly as good as the final one.

### Step 1: compute priors for the DPGMM 

To mimick a realistic scenario, in which SKYFAST runs in parallel with a PE algorithm, we cannot use all the samples to set the priors for the DPGMM. We found that using the first 500 samples is a good solution.  

We have to make sure that we define a parameter space that contains all the samples that we are going to pass to SKYFAST. The bounds for the angles is always well defined, while we have to be more careful for the luminosity distance. We found that a good solution is to set the luminosity distance bound to 1.5 times the maximum distance of the 500 samples that we pass to `get_priors`.

In [ ]:
np.random.shuffle(samples)
max_dist = np.max(np.array(samples[:500])[:,2])*1.5
bounds = np.array([[0., 2*np.pi], [-np.pi/2, np.pi/2], [0., max_dist], [0, np.pi]])
prior_pars = get_priors(bounds = bounds, samples = samples[:500], probit = True, scale=1)

### Step 2: initialise SKYFAST

Same as before, but now we set `entropy=True`.

In [ ]:
dens = skyfast(max_dist,                                   
                glade_file=glade_file,                     
                prior_pars= prior_pars,                    
                n_gridpoints=[800, 400, 100],              
                inclination = True,                        
                true_host=[3.44613, -0.40812, 41.00982],   
                true_inclination = [2.5831],                   
                out_name = 'gw170817_entropy_test',        
                out_folder= './results_tutorial',          
                theta_condition = True,       
                entropy = True  #we compute the information entropy each time a sample is added to the mixture              
                )

### Step 3: intermediate reconstuction 

As before, we use `intermediate_skymap` to pass the samples one by one to SKYFAST.

Since we set `entropy = True`, now SKYFAST computes the information entropy of the reconstructed posterior each time a sample is added to the DPGMM mixture.

When the information entropy reaches a plateau, adding further samples does not provide much additional information. This convergence criterion is usually met using a only fraction of the samples produced by the PE, resulting in considerable time savings when SKYFAST is run online alongside the PE pipeline.

After convergence is reached, the intermediate posterior reconstruction, along with the related skymap, volume map, and list of the most probable galaxy hosts, are released and saved in the output directory, marked as 'intermediate'.

In [ ]:
for i in tqdm(range(len(samples))):
        dens.intermediate_skymap(samples[i], show = True)

dens.density = dens.mix.build_mixture() 

### Step 4: entropy plots and comparison with final reconstruction 

Now that also the reconstruction using all the samples is finished, we can plot the entropy as a function of the number of samples in the mixture. 

We can also produce a corner plot of the posterior reconstructed posterior, its related skymap and volume map, to be compared with the intermediate ones. 

In [ ]:
# Entropy plot
dens.make_entropy_plot(show = True) 

# Reconstructed posterior, skymap and volume map obtained with all the PE samples
dens.density = dens.mix.build_mixture()
dens.save_density(final_map = True)
dens.plot_samples(samples, final_map = True, show = True)
dens.make_skymap(final_map = True, show = True)
dens.make_volume_map(final_map = True,  show = True)

# Save log file
dens.save_log() 

### Step 5: return to initial conditions

In [ ]:
dens.initialise()

## Recap of SKYFAST outputs

List of created directories and outputs. If `entropy = True`, the outputs are produced both for the intermediate and the final posterior reconsturction.

* CORNER: comparison of the original PE samples with samples from the reconstructed posterior
* DENSITY: file .json with the gaussian mixture
* ENTROPY: entropy plot
* HOSTS: list of galaxies within the 90% volume with or without the information about the inclination angle posterior conditioned to the position of each galaxies. 
* LOG: log file 
* SKYMAPS: skymap with credible regions and skymap with galaxies in the 90% credible volume
* VOLUME: volume map
